In [1]:
import psycopg2
import pandas as pd

DATABASE_URL="postgres://u2bbja15c4igeb:pedd96ce18167a13cc8d49f93dc8db09b97790d992dfcf068d16b08f4d0edbd0b@c4c161t4pf58h3.cluster-czrs8kj4isg7.us-east-1.rds.amazonaws.com:5432/d421pu93ggcr06"

def execute_sql(sql, return_all=False, commit=False):
    try:
        con = psycopg2.connect(DATABASE_URL, sslmode='require')
        cursor = con.cursor()
        cursor.execute(sql)
        
        if commit:
            con.commit()

        if return_all:
            out = cursor.fetchall()
            con.close()
            return out

        con.close()

    except psycopg2.OperationalError as e:
        try:
            print("Error:", e)
            con.close()
        except Exception:
            pass

# This query uses a window function to assign a row number per star rating.
# It then filters for the first 400 reviews per star rating.
query = """
SELECT user_id, business_id, stars, text, date
FROM (
    SELECT 
        user_id, 
        business_id, 
        stars, 
        text, 
        date,
        ROW_NUMBER() OVER (PARTITION BY stars ORDER BY date DESC) AS rn
    FROM reviews
) AS subquery
WHERE rn <= 400;
"""

# Execute the query and get the data
data = execute_sql(query, return_all=True)

# Convert the data into a DataFrame for easier manipulation
reviews_df = pd.DataFrame(data, columns=['user_id', 'business_id', 'stars', 'text', 'date'])
print(reviews_df.head())

reviews_df.to_pickle('reviews_df.pkl')

KeyboardInterrupt: 